---
title: Ion Scattering by Solar Wind Discontinuities
---

Check the properties related to ion scattering by solar wind discontinuities.


We need to use GLMakie because the volume is not supported by CairoMakie yet.

In [1]:
using Revise
using DrWatson
@quickactivate
include(srcdir("main.jl"))
includet(srcdir("main.jl"))
using Dates

In [2]:
df = load_wind();

┌ Warning: automatically converting Arrow.Timestamp with precision = MICROSECOND to `DateTime` which only supports millisecond precision; conversion may be lossy; to avoid converting, pass `Arrow.Table(source; convert=false)
└ @ Arrow /Users/zijin/.julia/packages/Arrow/rIDSw/src/eltypes.jl:273
┌ Warning: automatically converting Arrow.Timestamp with precision = NANOSECOND to `DateTime` which only supports millisecond precision; conversion may be lossy; to avoid converting, pass `Arrow.Table(source; convert=false)
└ @ Arrow /Users/zijin/.julia/packages/Arrow/rIDSw/src/eltypes.jl:273


In [3]:
Bn = :bn_over_b => abs => L"B_N/B";
v_norm = :v_to_norm => log10 => "Log v_gyro";

In [4]:
# velocity v is normalized to $l Ω$, where $l$ represents the width of the RD and $Ω = q B/(mpc)$ is the cyclotron frequency.
function v_norm!(df::AbstractDataFrame)
    df.v_to_norm = df.L_k .* df."B.mean"
end
v_norm!(df);

In [5]:
dfplot(layer, args...; axis = (;), kwargs...) = draw(layer * mapping(args...; kwargs...); axis)
dfplot(df::AbstractDataFrame, args...; axis = (;), kwargs...) = draw(data(df) * mapping(args...; kwargs...); axis)

dfplot (generic function with 2 methods)

In [6]:
xyz = (v_norm, Discontinuity.ω_in, Bn)

(:v_to_norm => (log10 => "Log v_gyro"), :ω_in => "in-plane rotation angle", :bn_over_b => (abs => L"$B_N/B$"))

In [7]:
using GLMakie

In [8]:
datalimits = ((1.5,5.5), (0,180), (0,1))
axis = (type = Axis3, width = 400)
plt = data(df) * mapping(xyz...; layout = :accuracy) * histogram(;datalimits, normalization=:pdf)
draw(plt; axis)
easy_save("3dhist"; formats=[:png], dir=figure_dir)

┌ Warning: Encountered an `AbstractVector` with value [1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.25, 5.75] on side x in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` instead.
└ @ Makie /Users/zijin/.julia/packages/Makie/6c4lt/src/conversions.jl:416
┌ Warning: Encountered an `AbstractVector` with value [5.0, 15.0, 25.0, 35.0, 45.0, 55.0, 65.0, 75.0, 85.0, 95.0, 105.0, 115.0, 125.0, 135.0, 145.0, 155.0, 165.0, 175.0, 185.0] on side y in `convert_arguments` for the `ImageLike` trait.
│         Using an `AbstractVector` to specify one dimension of an `ImageLike` is deprecated because `ImageLike` sides always need exactly two values, start and stop.
│         Use interval notation `start .. stop` or a two-element tuple `(start, stop)` ins